In [1]:
import torch
import torch.nn as nn
import nltk
from PIL import Image
from torch.autograd import Variable
from utils.build_vocab import Vocabulary
from model import EncoderCNN, VRNN
from utils.data_loader import get_loader
from collections import Counter
import pickle
from sys import path
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, PackedSequence
import numpy as np
from numpy.random import random_sample
%matplotlib inline

In [2]:
with open('data/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
encoder = EncoderCNN(256)
encoder.eval()
decoder = VRNN(256, 512, len(vocab), 512, 1)
if torch.cuda.is_available():
    encoder.cuda()
    decoder.cuda()

transform = transforms.Compose([
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])
rt_image = 'data/resized2014'
annotations = 'data/annotations/captions_train2014.json'
batch_size = 128
data_loader = get_loader(rt_image, annotations, vocab, transform, batch_size, True, 2)

/home/AD/irehg/anaconda2/envs/update-pytorch/lib/python2.7/site-packages/torch/backends/cudnn/__init__.py:48: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


data/annotations/captions_train2014.json
loading annotations into memory...
0:00:01.120238
creating index...
index created!


In [3]:
def decode_caption(caption):
    return ' '.join([vocab.idx2word[word_id] for word_id in caption.data.squeeze(0)])

In [4]:
def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

In [5]:
for i, (images, captions, lengths, _, _) in enumerate(data_loader):
    break

In [6]:
lengths = [l - 1 for l in lengths]
images = to_var(images, volatile=True)
captions = to_var(captions) # (1, 20(caption length))
targets = pack_padded_sequence(captions[:,1:], lengths, batch_first=True)[0]
decoder.zero_grad()
encoder.zero_grad()
features = encoder(images)

In [7]:
params = list(decoder.parameters())
optimizer = torch.optim.Adam(params, lr=1e-3)
def kl_div(q_mu, q_sigma, p_mu, p_sigma):
    return torch.log(p_sigma / q_sigma) + (q_sigma**2 + (p_mu - q_mu)**2) / 2 / p_sigma**2 - 0.5

In [8]:
prior, q_z, q_x = decoder(features, captions, lengths)

In [27]:
index = (torch.cuda.LongTensor(range(q_x.shape[0])), targets)

In [28]:
q_x[index]

Variable containing:
-9.3059
-9.1137
-9.0222
   ⋮   
-9.3737
-9.1748
-9.3439
[torch.cuda.FloatTensor of size 1549 (GPU 0)]